In [ ]:
# 1. Basic GAN (Vanilla GAN) -------------> Original Idea of GAN proposed by Ian Goodfellow
# 2. DCGAN (Deep Convolutional GAN)
#       - Uses CNN instead of ANN
#       - Better when it comes to creating Images
#
# 3. WGAN (Wasserstien GAN)
#       - Uses Wasserstien Distance formula instead of BCE loss
#       - More stable in terms of training
#
# 4. WGAN with Gradient Penalty (Analogy -- Applying Batch Normalization on all weights calculated)
#
# 5. Conditional GAN (cGAN)
#       - Generates data based on condition of label
#
# 6. AC-GAN ( Auxillary Classifier GAN)
#       - Good for multi-class image generation

In [ ]:
# Generator (for MNIST - 28x28 output)
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=1, feature_g=64):
        super().__init__()
        self.net = nn.Sequential(
            #                (inputDim, OutputChannel, kernelSize, stride, padding)
            # nn.ConvTranspose2d(100, 256, 3, 1, 0)
            nn.ConvTranspose2d(z_dim, feature_g * 4, 3, 1, 0),     # 1x1 → 3x3
            nn.BatchNorm2d(feature_g * 4),
            nn.ReLU(True),

            # nn.ConvTranspose2d(256, 128, 4,2,1)
            nn.ConvTranspose2d(feature_g * 4, feature_g * 2, 4, 2, 1),  # 3x3 → 7x7
            nn.BatchNorm2d(feature_g * 2),
            nn.ReLU(True),

             # nn.ConvTranspose2d(128, 64, 4,2,1)
            nn.ConvTranspose2d(feature_g * 2, feature_g, 4, 2, 1),      # 7x7 → 14x14   #OutputSize = (7-1)* 2 + 4-2*1 = 14
            nn.BatchNorm2d(feature_g),
            nn.ReLU(True),
             # nn.ConvTranspose2d(64, 1, 4,2,1)
            nn.ConvTranspose2d(feature_g, img_channels, 4, 2, 1),       # 14x14 → 28x28
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

# Discriminator (for MNIST - 28x28 input)
class Discriminator(nn.Module):
    def __init__(self, img_channels=1, feature_d=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, feature_d, 4, 2, 1),       # 28x28 → 14x14
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_d, feature_d * 2, 4, 2, 1),       # 14x14 → 7x7
            nn.BatchNorm2d(feature_d * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_d * 2, 1, 7, 1, 0),               # 7x7 → 1x1
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).view(-1, 1)